In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import cudf
import glob
import gc

from tqdm import tqdm

In [4]:
lb = False
path = '../../preprocess/'
    
df_type = cudf.DataFrame({
    'type': ['clicks', 'carts', 'orders'],
    'type_': [1, 2, 3]
})

In [5]:
!mkdir -p ./data/t4r/

mkdir: cannot create directory ‘./data/t4r/’: File exists


In [6]:
for seq_len in [20]:
    for split in [0]:
        print(seq_len, split)
        files = sorted(
            glob.glob(path + '/train/interim/*.parquet')
        ) + ['./data/xgb_train_x.parquet'] + ['../../data/test.parquet']

        df_aid = cudf.concat(
            [
                cudf.read_parquet(x, columns=['aid']) for x in sorted(files)
            ] 
        )
        df_aid = df_aid['aid'].value_counts().reset_index()
        df_aid = df_aid.reset_index()
        df_aid.columns = ['aid_', 'aid', 'count']
        df_aid['aid_'] = df_aid['aid_']+2
        
        files = sorted(
            glob.glob(path + '/train/interim/*.parquet')
        )

        df = cudf.concat(
            [
                cudf.read_parquet(x) for x in sorted(files)
            ] 
        ).reset_index(drop=True)
        if split>0:
            df = df.sort_values(['session', 'ts'], ascending=[True, False])
            df['ts_2'] = df.groupby(['session']).ts.shift(1)
            df['diff_'] = df['ts_2']-df['ts']
            df['diff_'] = ((df['diff_']/1000)>(split*60*60)).fillna(0).astype('int8')
            df['new_session'] = df.groupby(['session']).diff_.cumsum()
            df.drop(['ts_2', 'diff_'], axis=1, inplace=True)
        else:
            df['new_session'] = 0
        df = df.merge(
            df_type,
            how='left',
            on='type'
        )
        df = df.merge(
            df_aid[['aid_', 'aid']],
            how='left',
            on='aid'
        )
        df['aid_'] = df['aid_'].fillna(1)
        df['session'] = df['session'].astype('int32')
        df.drop(['type'], inplace=True, axis=1)
        df.drop(['aid'], inplace=True, axis=1)
        df = df.sort_values(['session', 'ts'], ascending=[True, False])
        df['dummy'] = 1
        df['rank'] = df.groupby(['session']).dummy.cumsum()
        df = df[df['rank']<=seq_len]
        df = df.groupby(['session', 'new_session']).agg({'aid_': list, 'type_': list, 'ts': 'min'})
        df = df.reset_index()
        df = df[df['aid_'].list.len()>1]
        df_train = df
        
        files = sorted(
            glob.glob('./data/xgb_train_x.parquet')
        )
        df = cudf.concat(
            [
                cudf.read_parquet(x) for x in sorted(files)
            ] 
        ).reset_index(drop=True)
        if split>0:
            df = df.sort_values(['session', 'ts'], ascending=[True, False])
            df['ts_2'] = df.groupby(['session']).ts.shift(1)
            df['diff_'] = df['ts_2']-df['ts']
            df['diff_'] = ((df['diff_']/1000)>(split*60*60)).fillna(0).astype('int8')
            df['new_session'] = df.groupby(['session']).diff_.cumsum()
            df.drop(['ts_2', 'diff_'], axis=1, inplace=True)
        else:
            df['new_session'] = 0
        df = df.merge(
            df_type,
            how='left',
            on='type'
        )
        df = df.merge(
            df_aid[['aid_', 'aid']],
            how='left',
            on='aid'
        )
        df['aid_'] = df['aid_'].fillna(1)
        df['session'] = df['session'].astype('int32')
        df.drop(['type'], inplace=True, axis=1)
        df.drop(['aid'], inplace=True, axis=1)
        df = df.sort_values(['session', 'ts'], ascending=[True, False])
        df['dummy'] = 1
        df['rank'] = df.groupby(['session']).dummy.cumsum()
        df = df[df['rank']<=seq_len]

        df = df.groupby(['session', 'new_session']).agg({'aid_': list, 'type_': list, 'ts': 'min'})
        df = df[df['aid_'].list.len()>1]
        df = df.reset_index()
        
        df.to_parquet('./data/t4r/t4r_xgb_train_x_' + str(seq_len) + '_' + str(split) + '.parquet')
        
        files = sorted(
            glob.glob('../../data/test.parquet')
        )
        df = cudf.concat(
            [
                cudf.read_parquet(x) for x in sorted(files)
            ] 
        ).reset_index(drop=True)
        if split>0:
            df = df.sort_values(['session', 'ts'], ascending=[True, False])
            df['ts_2'] = df.groupby(['session']).ts.shift(1)
            df['diff_'] = df['ts_2']-df['ts']
            df['diff_'] = ((df['diff_']/1000)>(split*60*60)).fillna(0).astype('int8')
            df['new_session'] = df.groupby(['session']).diff_.cumsum()
            df.drop(['ts_2', 'diff_'], axis=1, inplace=True)
        else:
            df['new_session'] = 0
        df = df.merge(
            df_type,
            how='left',
            on='type'
        )
        df = df.merge(
            df_aid[['aid_', 'aid']],
            how='left',
            on='aid'
        )
        df['aid_'] = df['aid_'].fillna(1)
        df['session'] = df['session'].astype('int32')
        df.drop(['type'], inplace=True, axis=1)
        df.drop(['aid'], inplace=True, axis=1)
        df = df.sort_values(['session', 'ts'], ascending=[True, False])
        df['dummy'] = 1
        df['rank'] = df.groupby(['session']).dummy.cumsum()
        df = df[df['rank']<=seq_len]

        df = df.groupby(['session', 'new_session']).agg({'aid_': list, 'type_': list, 'ts': 'min'})
        df = df.reset_index()
        df = df[df['new_session']==0]
        df.to_parquet('./data/t4r/t4r_xgb_sub_x_' + str(seq_len) + '_' + str(split) + '.parquet')
        print(df.shape)
        
        df_train = df_train.sort_values(['ts']).reset_index(drop=True)
        df_train.to_parquet('./data/t4r/t4r_train_' + str(seq_len) + '_' + str(split) + '.parquet')
        df_aid.to_parquet('./data/t4r/t4r_map_aid_' + str(seq_len) + '_' + str(split) + '.parquet')
        del df_train, df_aid, df
        gc.collect()

20 0
(1671803, 5)
